# Solar Tracker
Jupyter notebook for calculating the location of the sun for any date and time, 
given the observers location on the earths surface.

Before I became a software developer, I was a sculptor. I specialised in high-precision sundials, and traded under the name 'Precision Sundials'.

This was in the age before smartphones, and when measuring up for large public pieces, I often spent a long time with calculator, maps and a theodolite that I had purchased in an army surplus store. This meant that if I arrived on site on a sunny day with this kit, I could calculate True north to a fraction of a degree, something that cannot be done with a magnetic compass.

This notebook is a workbook I will use to refer to in creating the Android app that follows.
If smartphones had been available then, and if I had the programming skills that I have now, my life would have been so much easier...

## The position of earth relative to the sun

In this section, we take the line from sun to earth on new years day (Gregorian) as the origin. 

We calculate the angular distance that the earth has swept from this origin at the current date and time. 

This will be measured in radians for ease of calculation.

In [1]:
from datetime import datetime
import calendar
import math

In the section below, comment out the first or third line, depending on whether you want to use the current moment or to specify a time

In [2]:
now = datetime.now()
print('Actual Time', now)

now = datetime(2020, 1, 2, 12, 0, 0)
print('Selected Time', now)

Actual Time 2020-09-04 21:23:22.442839
Selected Time 2020-01-02 12:00:00


How many seconds in this year?

In [3]:
is_leap = calendar.isleap(now.year)
length_of_year = 366 if is_leap else 365
print('Number of days in this year', length_of_year)

# Mulitply number of days in year by number of seconds in a day
# to get number of seconds in a year.
length_of_year *= 24 * 60 * 60
print('Number of seconds in this year', length_of_year)

Number of days in this year 366
Number of seconds in this year 31622400


How many seconds have passed since noon on January first?

In [4]:
jan1 = datetime(now.year, 1, 1, 12, 0, 0)
seconds_elapsed = now.timestamp() - jan1.timestamp()
print(f'Seconds elapsed {seconds_elapsed}')

Seconds elapsed 86400.0


Express this in radians, giving the the angle off the sun relative to noon Jan 1

In [5]:
fraction = seconds_elapsed/length_of_year
print(f'As a fraction of a year: {fraction:.4f}')
print(f'In degrees:{fraction * 360:.1f}')

angle = fraction * 2 * math.pi
print(f'In radians:{angle:.4f}')

As a fraction of a year: 0.0027
In degrees:1.0
In radians:0.0172


## The Equation of Time

In this section, we make a small adjustment to the angle we just calculated. 
This corrects a slight discrepancy between clock and solar time. 

It takes the same values at any given date year after year. The maximum correction occurs on 3 November, and amounts to just under sixteen and a half minutes, meaning that solar time is ahead of clock time by about 16 minutes on this date every year. 

The correction varies as the year progresses, and is the same on any given calendar date year after year, give or take a few seconds.

For a good account of the Equation of Time, see https://en.wikipedia.org/wiki/Equation_of_time

This section can be ignored if you do not need a high-accuracy calculation for the position of the sun. For example, if you are simply measuring the length of a shadow, this correction will have little impact on the calculated length, and can be ignored.